# CH7

In [3]:
import keras
import tensorflow as tf

## 7.4

7-21 단계별 훈련 루프 작성하기: 훈련 스텝 함수

In [ ]:
model = get_mnist_model()

loss_fn = keras.losses.SparseCategoricalCrossentropy()  # 손실 함수 준비
optimizer = keras.optimizers.RMSprop()  # 옵티마이저 준비
metrics = [keras.metrics.SparseCategoricalAccuracy()]  # 모니터링할 지표 리스트 준비
loss_tracking_metric = keras.metrics.Mean()  # 손실 평균을 추적할 평균 지표 준비

def train_step(inputs, targets):
  # 정방향 패스를 실행
  with tf.GradientTape() as tape:
    predictions = model(inputs, training=True)
    loss = loss_fn(targets, predictions)
  # 역방향 패스 실행
  gradients = tape.gradient(loss, model.trainable_weights)
  optimizer.apply_gradients(zip(gradients, model.trainable_weights))
  # 측정 지표 계산
  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs[metric.name] = metric.result()
  # 손실 평균 계산
  loss_tracking_metric.update_state(loss)
  logs["loss"] = loss_tracking_metric.result()
   # 지표와 손실의 현재 값을 반환
  return logs

7-22 단계별 훈련 루프 작성하기: 지표 재설정

In [2]:
def reset_metrics():
  for metric in metrics:
    metic.reset_state()
  loss_tracking_metric.reset_index()

7-23 단계별 훈련 루프 작성하기: 훈련 루프 자체

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images, train_labels))
traing_dataset = training_dataset.batch(32)
epochs = 3
for epoch in range(epochs):
  reset_metrics()
  for inputs_batch, targets_batch in training_dataset:
    logs = train_step(inputs_batch, targets_batch)
  print(f"{epoch}번째 에포크 결과")
  for key, value in log.items():
    print(f"...{key}: {value:.4f}")

7-24 단계별 평가 루프 작성하기

In [ ]:
def test_step(inputs, targets):
  predictions = model(inputs, training=False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs["val_" + metric.name] = metric.result()
  loss_tracking_metric.update_state(loss)
  logs["val_loss"] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for input_batch, targets_batch in val_dataset:
  logs = test_stop(inputs_batch, targets_batch)
print("평가 결과: ")
for key, value in logs.items():
  print(f"...{key}: {value:.4f}")

7-25 평가 스텝 함수에 @tf.function 데코레이터 추가하기

In [ ]:
@tf.function
def test_step(inputs, targets):
  predictions = model(inputs, training=False)
  loss = loss_fn(targets, predictions)

  logs = {}
  for metric in metrics:
    metric.update_state(targets, predictions)
    logs["val_" + metric.name] = metric.result()
  loss_tracking_metric.update_state(loss)
  logs["val_loss"] = loss_tracking_metric.result()
  return logs

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.batch(32)
reset_metrics()
for input_batch, targets_batch in val_dataset:
  logs = test_stop(inputs_batch, targets_batch)
print("평가 결과: ")
for key, value in logs.items():
  print(f"...{key}: {value:.4f}")

7-26 fit()이 사용할 사용자 저의 훈련 스텝 구현하기

In [ ]:
loss_Fn = keras.losses.SpaseCategorcialCrossentropy()
loss_tracker = keras.metrics.Mean(name="loss")

class CustomModel(keras.Model):
  def train_step(self, data):
    inputs, targets = data
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = loss_fn(targets, predictions)
    gradients = tape.gradient(loss, self.trainable_weights)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
    loss_tracker.update_state(loss)
    return{"loss" : loss_tracker.result()}

    @property
    def metrics(self):
      return [loss_tracker]